In [ ]:
# Nikhil Patil
# CSEC 620 Project 2
# K-Nearest Neighbor Clustering Implementation 

from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import re, random
from sklearn.preprocessing import StandardScaler

class KNearestNeighbor:
    
    def __init__(self):
        
        